In [52]:
import glob

ignoreFiles= ["tags.md", "index.md","PLACEHOLDER.md"]

docClasses = {}

for filepath in glob.iglob('../../docs/*.md'):
    trimFileName = filepath.split("\\")[len(filepath.split("\\"))-1]
    if trimFileName in ignoreFiles:
        continue

    className = trimFileName.replace(".md","")
    docClasses[className] = []

    file = open(filepath, 'r')
    docLines = file.readlines()
    for line in docLines:
        if line.strip().startswith("###") and not line.strip().startswith("####") and "aria-label" in line:
            filteredLine = line.replace("#","").replace("Â·","").replace("()","").split("{")[0].strip()
            docClasses[className].append(filteredLine)


In [53]:
file2 = open('api_dump.dat', 'r')
bindsLines = file2.readlines()

nameReplace = {
    "APIOverride.OverriddenClasses.Level.Original.__const": "Level",
    "ItemConfig.Config": "ItemConfig"
}

classes = {}

curClass= ""
curClassLVL= 0
for line in bindsLines:
    if not line.strip():
        continue
    leadingTabs = len(line) - len(line.lstrip())
    if line.lstrip().startswith("Class"):
        className = line.lstrip().split(" ")[1]
        if className in nameReplace:
            className = nameReplace[className]
        className = className.replace(".","_") # make it doc complient
        classes[className] = []
        curClass = className
        curClassLVL = leadingTabs
    elif line.lstrip().startswith("Table") or line.lstrip().startswith("Subclasses") :
        continue
    else:
        if curClass and curClassLVL < leadingTabs:
            filterFunc = line.strip().split(" ")[0].split(":")[0]
            classes[curClass].append(filterFunc)

### Classes and functions that are not documented, but were found by the current automatic api grabber tool

In [ ]:
for dumpClass in classes:
    if dumpClass not in docClasses:
        print("NOT DOCUMENTED CLASS:", dumpClass)
    else:
        docClassContents = docClasses[dumpClass]
        dumpClassContents = classes[dumpClass]
        for dumpContent in dumpClassContents:
            if dumpContent not in docClassContents:
                print("NOT DOCUMENTED CONTENT IN CLASS:",dumpClass,"\t", "CONTENT:",dumpContent)

### Classes and functions that are documented, but not found by the current automatic api grabber tool

In [ ]:
for docClass in docClasses:
    if docClass not in classes:
        print("NOT CAPTURED CLASS:", docClass)
    else:
        docClassContents = docClasses[docClass]
        dumpClassContents = classes[docClass]
        for content in docClassContents:
            if content not in dumpClassContents:
                print("NOT CAPTURED CONTENT IN CLASS:",docClass,"\t", "CONTENT:",content)